In [1]:
import os
import gc 
from glob import glob
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import lightgbm as lgb
import torch
import torch.nn as nn

from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier, Pool
import xgboost
from collections import Counter

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
ROOT            = Path("/kaggle/input/home-credit-risk-model-train-test")
SCHEMA_PATH            = Path("/kaggle/input/schema-home-credit-risk-data")

In [3]:
def set_data_types(df, schema):
    
    for col, dtype in schema.items():
        if dtype == 'category':
            dtype = 'object'
        df[col] = df[col].astype(dtype)
    return df
    

In [4]:
X = pl.read_csv(ROOT/'model_abt_train.csv').to_pandas()
schema = pd.read_csv(SCHEMA_PATH / "data_schema.csv", names = ['Columns', 'dtype'])
schema = schema[schema['Columns'].isin(X.columns)]
schema = schema.set_index('Columns')['dtype'].to_dict()
X = set_data_types(X, schema)
del schema

In [5]:
id_val = X[['case_id', 'WEEK_NUM']]
y = X['target']
cols = X.drop(['case_id', 'WEEK_NUM', 'target', 'decision_month', 'decision_weekday'], axis = 1).columns
X = X.drop(['case_id', 'WEEK_NUM', 'target', 'decision_month', 'decision_weekday'], axis = 1)

In [6]:
stats = X.describe().T
stats['perc_nulls'] = stats['count']/len(X)
drop = stats[stats['perc_nulls']>0.95].index
print(drop)
X = X.drop(drop, axis = 1)
del stats
del drop
gc.collect()

Index(['annuity_780A', 'annuitynextmonth_57A', 'applicationcnt_361L',
       'applications30d_658L', 'applicationscnt_1086L', 'applicationscnt_464L',
       'applicationscnt_629L', 'applicationscnt_867L',
       'clientscnt12m_3712952L', 'clientscnt_100L', 'clientscnt_1022L',
       'clientscnt_1071L', 'clientscnt_1130L', 'clientscnt_157L',
       'clientscnt_257L', 'clientscnt_304L', 'clientscnt_360L',
       'clientscnt_493L', 'clientscnt_533L', 'clientscnt_887L',
       'clientscnt_946L', 'credamount_770A', 'currdebtcredtyperange_828A',
       'disbursedcredamount_1113A', 'downpmt_116A', 'homephncnt_628L',
       'mobilephncnt_593L', 'numactivecreds_622L',
       'numactivecredschannel_414L', 'numactiverelcontr_750L',
       'numcontrs3months_479L', 'numinstls_657L', 'numnotactivated_1143L',
       'numpmtchanneldd_318L', 'numrejects9m_859L', 'pmtnum_254L',
       'sellerplacecnt_915L', 'sellerplacescnt_216L', 'totalsettled_863A',
       'first_birth_259D', 'mainoccupationinc_384A_m

0

In [7]:
from collections import Counter
Counter(X.dtypes)

Counter({dtype('float64'): 327, dtype('O'): 81, dtype('bool'): 1})

In [8]:
cat_cols = [x for x, dtypes in X.dtypes.items() if dtypes == 'O']
num_cols = [x for x, dtypes in X.dtypes.items() if dtypes in ['float', 'int']]

In [9]:
#cat_cols = [col for col, dtype in dict(X.dtypes).items() if dtype == 'object']
#X[cat_cols] = X[cat_cols].astype('category')

In [10]:
score_schema = {'model_name': 'str', 
                        'model': 'str',
                        'params': 'str',
                        'acc': 'float',
                        'precision': 'float',
                        'recall': 'float',
                        'f1': 'float',
                        'roc_auc': 'float',
                        'pr_auc': 'float',
                        'lift': 'float'}
scores = pd.DataFrame(columns = score_schema.keys()).astype(score_schema)
scores

,model_name,model,params,acc,precision,recall,f1,roc_auc,pr_auc,lift


In [11]:
class Model_Utils:
    @staticmethod
    def model_evals(y_true, y_proba, cutoff = 0.5):
        from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, average_precision_score, roc_auc_score
        """
        Returns model evaluation metrics for a binary classification model

        Parameters:
        -----------
            y_true: int (0,1) 
                Actual binary labels

            y_proba: float (between 0 and 1)
                Probability scores output of model 

        Returns:
        --------
            result: dict
                Dictionary of metrics and their results based on the input
                    - event rate (% predicted 1's)
                    - accuracy
                    - roc_auc
                    - pr_auc
                    - recall
                    - precision
                    - f1
                    - lift
        """

        y_pred = (y_proba > cutoff).astype(int)

        event_rate = y_pred.mean()

        accuracy = accuracy_score(y_true, y_pred)

        roc_auc = roc_auc_score(y_true, y_proba)

        pr_auc = average_precision_score(y_true, y_proba)

        recall = recall_score(y_true, y_pred)

        precision = precision_score(y_true, y_pred)

        f1 = f1_score(y_true, y_pred)

        lift = recall / event_rate

        return {'event_rate': event_rate,
                'acc': accuracy, 
                'precision': precision, 
                'recall': recall, 
                'f1': f1, 
                'roc_auc': roc_auc,
                'pr_auc': pr_auc, 
                'lift': lift}
    
    @staticmethod
    def DumbClassifier(y_true):
        import numpy as np
        y_pred = np.zeros(len(y_true))
        return y_pred
    
    @staticmethod
    def RandomChanceClassifier(y_true):
        import numpy as np
        y_pred = Model_Utils.DumbClassifier(y_true)
        event = y_true.sum()
        ind = np.random.randint(0, len(y_true), size=event)
        y_pred[ind] = 1
        return y_pred
    
    @staticmethod
    def save_model_results(results, schema, filepath):
        if not os.path.exists(filepath):
            score_schema = {'model_name': 'str', 
                        'model': 'str',
                        'params': 'str',
                        'acc': 'float',
                        'precision': 'float',
                        'recall': 'float',
                        'f1_score': 'float',
                        'roc_auc': 'float',
                        'pr_auc': 'float',
                        'lift': 'float'}
            scores = pd.DataFrame(columns = score_schema.keys()).astype(score_schema)
        else:
            scores = pd.read_csv(filepath)
        row = []
        
        for key in schema.keys():
            row.append(results[key])
        
        scores.loc[len(scores)] = row
        scores.to_csv(filepath, index = False)
        
    @staticmethod
    def LightGBMClassifier_CV(X, y, cat_cols, cv = 5, group = None, params = None):
        import lightgbm
        from sklearn.model_selection import StratifiedGroupKFold
        
        cv = StratifiedGroupKFold(n_splits=cv, shuffle = True, random_state = 42)
        
        if params == None:
            params = {'random_state': 42
                     ,'objective': 'binary'
                     ,'verbose': -1
                     ,'n_jobs': -1}
        
        scores = {'acc': [],
                        'precision': [],
                        'recall': [],
                        'f1': [],
                        'roc_auc': [],
                        'pr_auc': [],
                        'lift': []}

        split = 1
        
        for train_ind, valid_ind in cv.split(X, y, groups=group):
            X_train, y_train = X.iloc[train_ind], y.iloc[train_ind]
            X_valid, y_valid = X.iloc[valid_ind], y.iloc[valid_ind]

            X_train[cat_cols] = X_train[cat_cols].astype("category")
            X_valid[cat_cols] = X_valid[cat_cols].astype("category")
            
            lgb = lightgbm.LGBMClassifier(**params)
            lgb.fit(X_train, y_train)

            y_proba = lgb.predict_proba(X_valid)[:, 1]
            results = Model_Utils.model_evals(y_valid, y_proba)
            print(f'LightGBM, Val CV{split}: {results}"')
            
            split += 1
            
            for key in results.keys():
                if key in scores.keys():
                    scores[key].append(results[key])
        
        for key in results.keys():
            if key in scores.keys():
                scores[key] = np.array(scores[key]).mean()
            
        print(f'LightGBM, Average CV: {scores}"')
        return lgb, scores

    
    @staticmethod
    def train_test_split(X, y, test_size = 0.2):
        from sklearn.model_selection import train_test_split
        import polars as pl
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
        pl.from_pandas(pd.concat([y_train, X_train], axis = 1)).write_csv('model_abt_train.csv')
        del X_train
        del y_train
        gc.collect()
        
        pl.from_pandas(pd.concat([y_test, X_test], axis = 1)).write_csv('model_abt_test.csv')
        del X_test
        del y_test
        gc.collect()

In [12]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
#from utils.model_helpers import model_evals
import xgboost
import lightgbm
from time import time
from sklearn.model_selection import cross_validate, StratifiedKFold
import numpy as np

space = {
    'max_depth': hp.quniform("max_depth", 3, 10, 1),  #Max depth 10
    'gamma': hp.loguniform('gamma', np.log(1), np.log(100)),
    'reg_alpha': hp.uniform('reg_alpha', 0, 10),
    'reg_lambda': hp.uniform('reg_lambda', 0, 10),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'min_child_weight': hp.loguniform('min_child_weight',np.log(1), np.log(100)),
    'n_estimators': hp.quniform('n_estimators', 50, 250, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'random_state': 42,   
    'verbose': -1
}

def objective(space):
    start_time = time()

    print(f"Parameters: {space}")
    space['max_depth'] = int(space['max_depth'])
    #space['reg_alpha'] = int(space['reg_alpha'])
    space['n_estimators'] = int(space['n_estimators'])

    #cv_folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    #i = 1
    
    model_name = f'LGB_Tune{len(scores)}'
    model_type = 'LightGBM'
    params = space
    lgb, dict_results = Model_Utils.LightGBMClassifier_CV(X, y, cat_cols, group = id_val['WEEK_NUM'], params = space)  
    
    #for train_index, test_index in cv_folds.split(X[num_cols], y):

    #    print(f"CV Fold: {i}")
        # Get the training and testing data for this fold
    #    X_train_cv, X_test_cv = X[num_cols].iloc[train_index], y.iloc[test_index]
    #    y_train_cv, y_test_cv = X[num_cols].iloc[train_index], y.iloc[test_index]
        
    #    lgb = lightgbm.LGBMClassifier(**space)
    #    lgb.fit(X_train_cv, y_train_cv)

    #    y_test_proba_cv = lgb.predict_proba(X_test_cv)[:, 1]
    #    results = Model_Utils.model_evals(y_test_cv, y_test_proba_cv)
    #    print(f'LightGBM, Val CV{i} Tuned: {results}" ')

    #    i += 1
        
    #    for key in dict_results.keys():
    #        dict_results[key].append(results[key])
    
    #lgb.fit(X_train_val, y_train_val)
    #y_test_proba = lgb.predict_proba(X_test)[:, 1]
    #test_results = model_evals(y_test, y_test_proba)
    #print(f'LigthGBM, OOT Tuned: {test_results}" ')

    elapsed_time = time() - start_time
    
    row = [model_name, model_type, params]
    #for key in dict_results.keys():
    #    row.append(test_results[key])

    for key in dict_results.keys():
        row.append(dict_results[key])

    
    print(row)
    scores.loc[len(scores)] = row
    scores.to_csv('LGB_Tuned_Mobile_Train.csv')

    return {'loss': -dict_results['pr_auc'], 'status': STATUS_OK, 'model': lgb, 'elapsed_time': elapsed_time}

In [13]:
trials = Trials()
best = fmin(fn = objective, space = space, algo = tpe.suggest, max_evals = 250, trials = trials)

  0%|          | 0/250 [00:00<?, ?trial/s, best loss=?]

Parameters: {'colsample_bytree': 0.6589389917537107, 'gamma': 1.3997730991483204, 'learning_rate': 0.016892867554590824, 'max_depth': 8.0, 'min_child_weight': 23.55079762531943, 'n_estimators': 112.0, 'random_state': 42, 'reg_alpha': 4.409710014747803, 'reg_lambda': 1.3538506979609166, 'subsample': 0.9698048241955275, 'verbose': -1}
  0%|          | 0/250 [00:00<?, ?trial/s, best loss=?]

LightGBM, Val CV1: {'event_rate': 0.0, 'acc': 0.9695184217898551, 'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'roc_auc': 0.8288370729073808, 'pr_auc': 0.1581516965537917, 'lift': nan}"
  0%|          | 0/250 [03:02<?, ?trial/s, best loss=?]

LightGBM, Val CV2: {'event_rate': 0.0, 'acc': 0.9697996221599727, 'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'roc_auc': 0.8153199330326104, 'pr_auc': 0.1420285521954761, 'lift': nan}"
  0%|          | 0/250 [05:54<?, ?trial/s, best loss=?]

LightGBM, Val CV3: {'event_rate': 0.0, 'acc': 0.9652123642655585, 'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'roc_auc': 0.8052281533798238, 'pr_auc': 0.15531227875575757, 'lift': nan}"
  0%|          | 0/250 [08:49<?, ?trial/s, best loss=?]

...

 14%|█▍        | 35/250 [11:35:15<76:15:23, 1276.85s/trial, best loss: -0.18499271414426305]

Parameters: {'colsample_bytree': 0.8077266991312345, 'gamma': 1.0269687323388519, 'learning_rate': 0.08316421730631791, 'max_depth': 7.0, 'min_child_weight': 12.731033425003735, 'n_estimators': 250.0, 'random_state': 42, 'reg_alpha': 7.013026939752677, 'reg_lambda': 9.268151666956935, 'subsample': 0.5629669911383062, 'verbose': -1}
 14%|█▍        | 35/250 [11:35:15<76:15:23, 1276.85s/trial, best loss: -0.18499271414426305]

LightGBM, Val CV1: {'event_rate': 0.000849471791210678, 'acc': 0.9695636064596004, 'precision': 0.526595744680851, 'recall': 0.014675363178179662, 'f1': 0.02855494663974618, 'roc_auc': 0.8599741552972002, 'pr_auc': 0.19725388224431908, 'lift': 17.275868757530073}"
 14%|█▍        | 35/250 [11:40:50<76:15:23, 1276.85s/trial, best loss: -0.18499271414426305]

LightGBM, Val CV2: {'event_rate': 0.0007970661833175581, 'acc': 0.9698685990412214, 'precision': 0.5432692307692307, 'recall': 0.014338281943915747, 'f1': 0.02793917665966127, 'roc_auc': 0.8474345829851017, 'pr_auc': 0.17264710538693184, 'lift': 17.988822313646256}"
 14%|█▍        | 35/250 [11:46:11<76:15:23, 1276.85s/trial, best loss: -0.18499271414426305]

LightGBM, Val CV3: {'event_rate': 0.0010915353455315778, 'acc': 0.9654064149936529, 'precision': 0.5888888888888889, 'recall': 0.018477629285299245, 'f1': 0.035830985915492955, 'roc_auc': 0.8415488812558779, 'pr_auc': 0.19643312189263293, 'lift': 16.928108980566854}"
 14%|█▍        | 35/250 [11:51:32<76:15:23, 1276.85s/trial, best loss: -0.18499271414426305]

In [ ]:
params = {'colsample_bytree': 0.7293329587571634, 'gamma': 7.214695665698402, 'learning_rate': 0.017453441223973187, 'max_depth': 8.0, 'min_child_weight': 15.172422243573173, 'n_estimators': 151.0, 'random_state': 42, 'reg_alpha': 6.601119741508825, 'reg_lambda': 6.3778378824017175, 'subsample': 0.7810209900884229, 'verbose': -1}

In [ ]:
results  = Model_Utils.LightGBMClassifier_CV(X, y, cat_cols, group = id_val['WEEK_NUM'])
results['model_name'] = 'LightGBM Default Classifier'
results['model'] = 'LigthGBM'
Model_Utils.save_model_results(results, score_schema, "baseline.csv")
del results
gc.collect()